In [3]:
import os
import pickle
import scipy.stats
import gzip
import requests
import sys
import json
import io
import numpy as np
import re
import urllib
import StringIO
import csv
import pandas as pd
os.chdir("/Users/leobrueggeman/GitHub/LINCS/l1ktools/python")
import cmap.io.gct as gct
import cmap.io.plategrp as grp
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import itertools

In [5]:
# function that takes gzipped github tsv files and creates dataframe
def url_to_df(path):
    url = urllib.urlopen(path)
    url_f = StringIO.StringIO(url.read())
    g = gzip.GzipFile(fileobj=url_f)
    return pd.read_table(g)

In [6]:
# set path to LINCS gctx data
path_to_gctx_file = '/Volumes/HimmelWD/lincs/modzs.gctx'

### Load in various mappings (perturbagen, signature, gene)

In [7]:
# construct gene_df
path = 'https://github.com/dhimmel/lincs/raw/d42347fcb53c30afed705b973fb52a1ae45a26b1/data/geneinfo/geneinfo.tsv.gz'
gene_df = url_to_df(path)
gene_df.dropna(inplace=True)
gene_df.reset_index(drop=True, inplace=True)

In [238]:
gene_df.head()

,pr_id,pr_gene_id,pr_gene_symbol,pr_gene_title,is_lm,is_l1000,is_bing
0,202938_x_at,100510314///100510451///27341///91695,-666,-666,False,True,False
1,204006_s_at,2214///2215,-666,-666,False,True,False
2,204060_s_at,5613///5616,-666,-666,False,True,False
3,204419_x_at,3047///3048,-666,-666,False,True,False
4,204438_at,414308///4360,-666,-666,False,True,False


In [8]:
# construct pert_df
path = 'https://github.com/dhimmel/lincs/raw/d42347fcb53c30afed705b973fb52a1ae45a26b1/data/pertinfo/pertinfo.tsv.gz'
pert_df = url_to_df(path)
pert_df = pert_df[['pert_id', 'pubchem_cid']]
pert_df.columns=['pert_id', 'pubchem_id']
pert_df.dropna(inplace=True)
pert_df.pubchem_id = pert_df['pubchem_id'].astype(str)
pert_df.reset_index(drop=True, inplace=True)

In [239]:
pert_df.head()

,pert_id,pubchem_id
0,BRD-K68741898,44505553
1,BRD-A05457250,2303
2,BRD-K72034655,8616
3,BRD-K02458594,21785456
4,BRD-K18814832,2228302


In [10]:
# construct sig_df
path = 'https://github.com/dhimmel/lincs/raw/d42347fcb53c30afed705b973fb52a1ae45a26b1/data/siginfo/siginfo.tsv.gz'
sig_df = url_to_df(path)
sig_df = sig_df[sig_df.is_gold == True] 
sig_df = sig_df[['sig_id', 'pert_id', 'pert_itime', 'pert_idose', 'cell_id']]

In [240]:
sig_df.head()

,sig_id,pert_id,pert_itime,pert_idose,cell_id
1,CVD001_HUH7_24H:BRD-K07762753-001-03-6:50,BRD-K07762753,24 h,50 µM,HUH7
12,CPC004_VCAP_6H:BRD-A46393198-003-10-9:10,BRD-A46393198,6 h,10 µM,VCAP
16,CPC005_VCAP_6H:BRD-A47494775-003-03-0:10,BRD-A47494775,6 h,10 µM,VCAP
21,CPC005_VCAP_6H:BRD-A09925278-003-03-1:10,BRD-A09925278,6 h,10 µM,VCAP
22,CPC005_VCAP_6H:BRD-A18419789-001-01-4:10,BRD-A18419789,6 h,10 µM,VCAP


In [11]:
# load and process drugbank to pubchem matching file
path = 'https://github.com/dhimmel/drugbank/raw/e8567eed2dd48ae0694a0960c518763a777845ff/data/mapping/pubchem.tsv'
drugbank_df = pd.read_table(path)
drugbank_df.columns=['drugbank_id', 'pubchem_id']
drugbank_df.pubchem_id = drugbank_df['pubchem_id'].astype(str)

In [241]:
drugbank_df.head()

,drugbank_id,pubchem_id
0,DB00014,11980055
1,DB00014,11981235
2,DB00014,11982741
3,DB00014,16052011
4,DB00014,23581804


### Create probe lists, to be used when querying GCTX files

In [245]:
# create csv files of is_bing and is_lm probe to gene id 
is_bing = gene_df[gene_df.is_bing == True]
is_bing = is_bing[['pr_id', 'pr_gene_id']]
bing_list = list(is_bing.pr_id)
is_lm = gene_df[(gene_df.is_lm == True) & (gene_df.is_bing == True)]
is_lm = is_lm[['pr_id', 'pr_gene_id']]
lm_list = list(is_lm.pr_id)

### Create master metadata dataframe (total_df) containing all metadata that we are interested in

In [13]:
# create dataframe with all organized, mapped meta-data
total_df = drugbank_df.merge(pert_df, how='inner')
total_df = total_df.merge(sig_df, how='inner')

In [19]:
total_df.head()

,drugbank_id,pubchem_id,pert_id,sig_id,pert_itime,pert_idose,cell_id
0,DB00014,23581804,BRD-A62434282,CPC010_MCF7_24H:BRD-A62434282-015-02-8:10,24 h,10 µM,MCF7
1,DB00091,16404350,BRD-A38030642,CPC006_A375_24H:BRD-A38030642-001-02-0:10,24 h,10 µM,A375
2,DB00091,16404350,BRD-A38030642,CPC001_PC3_24H:BRD-A38030642-001-01-2:10,24 h,10 µM,PC3
3,DB00091,16404350,BRD-A38030642,CPC006_A549_24H:BRD-A38030642-001-02-0:10,24 h,10 µM,A549
4,DB00091,16404350,BRD-A38030642,CPC006_HCC15_6H:BRD-A38030642-001-02-0:10,6 h,10 µM,HCC15


In [14]:
# This is the list of all gold signatures we are interested in (belong to perturbagens which map to drugbank drugs)
total_sig = list(set(total_df.sig_id))
total_sig.sort()

### Functions to interact, combine, and process data extracted from LINCS GCTX file

In [135]:
def pert_to_cid(pert_id_here):
    """
    give pert_id's and receive corresponding sig_id_gold's, if ref_dict exists
    """
    column_ids = ref_dict[pert_id_here]['sig_id_gold']
    return column_ids
        
def cid_to_matrix(cid_here, rid_here):
    """
    give cid(signature ids) and rid(probe_ids) and get back expression matrix
    """
    GCTObject = gct.GCT(path_to_gctx_file)
    GCTObject.read_gctx_matrix(cid = cid_here, rid = rid_here)
    pert_matrix = GCTObject.matrix
    return pert_matrix

def cid_to_expr_df(cid_here, rid_here):
    """
    give cid(signature ids) and rid(probe_ids) and get back expression dataframe with row and column names
    """
    GCTObject = gct.GCT(path_to_gctx_file)
    GCTObject.read_gctx_matrix(cid = cid_here, rid = rid_here)
    pert_matrix = GCTObject.matrix
    expr_df = pd.DataFrame(pert_matrix, index=rid_here, columns=cid_here)
    return expr_df

def expr_df_to_expr_matrix(expr_df, cid_here, rid_here):
    """
    give expr_df, cid(signature ids) and rid(probe_ids) and get back a subset of your expr_df which has
    only the values of the desired signatures (cid) and probes (rid) in numpy array form
    """
    return expr_df[cid_here][rid_here].as_matrix

def matrix_to_corr_matrix(matrix_here):
    """
    give expression matrix, and get spearman correlation matrix (numpy array)
    do not pass this expression matrices which only have 1 or 2 signatures
    """
    corr_matrix = scipy.stats.spearmanr(matrix_here)
    corr_matrix_2 = corr_matrix[0]
    return corr_matrix_2


def corr_matrix_to_sig_corr_value_and_total_corr(corr_matrix):
    """
    give correlation matrix (numpy array) and get list of signature correlation values and total correlation value.
    signature correlation value is the signature-wise mean correlation value of one signature to the rest of the
    signatures.
    signature correlation values below 0.1, are set to 0.1, so that no signature may contribute a negative amount
    to the consensus matrix
    """
    output_list = []
    temp_len = len(corr_matrix)
    for i in range(temp_len):
        temp_sum = corr_matrix[:,i].sum() - 1
        temp_mean = temp_sum / (temp_len - 1)
        output_list.append(temp_mean)
    for i in range(len(output_list)):
        if output_list[i] < 0.05:
            output_list[i] = 0.05
    total_corr = np.mean(output_list)
    output_list[:] = [x/sum(output_list) for x in output_list]
    return output_list, total_corr

def sig_corr_value_to_expr_sig(matrix_here, sig_corr_value):
    """
    give signature correlation value and get weighted consensus expression signature
    """
    temp_len = matrix_here.shape[1]
    matrix_here *= sig_corr_value
    matrix_here = np.sum(matrix_here, axis=1)
    return matrix_here.tolist()
    

### Create master expression dataframe (total_expr_df) with expression data for all signatures we are interested in across entire is_bing probe set

In [16]:
total_expr_df = cid_to_expr_df(total_sig, list(is_bing.pr_id))

In [93]:
total_expr_df.head()


,AML001_CD34_24H:BRD-A19037878:1.11111,AML001_CD34_24H:BRD-A19037878:3.33333,AML001_CD34_24H:BRD-A19500257:0.37037,AML001_CD34_24H:BRD-A19500257:1.11111,AML001_CD34_24H:BRD-A19500257:10,AML001_CD34_24H:BRD-A19500257:3.33333,AML001_CD34_24H:BRD-A75409952:0.37037,AML001_CD34_24H:BRD-A75409952:10,AML001_CD34_24H:BRD-A75409952:3.33333,AML001_CD34_24H:BRD-K19706299:0.37037,...,RAD001_PC3_6H:BRD-K81418486-001-18-6:3.3333,RAD001_PC3_6H:BRD-K81418486:10,RAD001_PC3_6H:BRD-K83896451-001-02-6:0.0137,RAD001_PC3_6H:BRD-K84937637-001-03-2:0.0412,RAD001_PC3_6H:BRD-K84937637-001-03-2:0.3704,RAD001_PC3_6H:BRD-K84937637-001-03-2:1.1111,RAD001_PC3_6H:BRD-K84937637-001-03-2:10,RAD001_PC3_6H:BRD-K84937637-001-03-2:3.3333,RAD001_PC3_6H:BRD-K97530723-001-11-8:0.0046,RAD001_PC3_6H:BRD-K97530723-001-11-8:10
218075_at,1.74245,0.54680,0.50055,0.40590,0.81145,0.51100,-0.18800,0.17875,1.09800,1.26385,...,-0.6707,-1.21900,-0.87485,1.11535,-0.22760,0.23965,-0.35225,0.08490,-1.46285,-0.95135
218434_s_at,1.79360,1.90810,-1.33360,0.55185,-1.37750,-1.13700,0.48815,0.81535,-1.39535,0.53570,...,1.9232,0.24905,-0.92480,-0.25745,-1.58210,-0.87280,-0.18620,-0.15370,0.56200,1.62305
202852_s_at,-0.11780,2.01105,-1.53820,-1.42820,-1.69210,-0.64800,0.19510,-0.76880,0.39220,0.56270,...,-0.4731,-0.17735,-0.93560,0.41505,-0.85950,-2.06780,-1.33420,-0.93005,1.17325,1.24850
205434_s_at,-0.49925,-0.76745,0.61830,1.86725,1.26745,0.46705,-0.05310,-0.29585,-1.25695,-0.59115,...,0.9660,1.34635,0.04875,-1.05025,-1.49765,1.20005,0.16370,0.69865,-0.15870,1.09740
201511_at,1.94740,3.13170,-0.22405,0.23915,1.17865,0.88555,0.29595,0.38090,0.83415,1.04595,...,0.5911,-2.69275,-0.22285,0.83150,-0.76900,-1.26455,-1.14485,-2.19495,-0.56505,0.89965


###Create dataframe (cons_expr_bing_df) containing consensus expression signatures of drugbank drugs by bing lists

In [141]:
# create ref_dic which contains drugbank_ids and their associated signatures
ref_dic = {}
for index, row in total_df.iterrows():
    temp_drug = row['drugbank_id']
    temp_sig = row['sig_id']
    temp_dic = ref_dic.setdefault(temp_drug, [])
    temp_dic.append(temp_sig)
    
# create consensus dataframe which we will fill iteratively
cons_expr_bing_df = pd.DataFrame(np.nan, index=[], columns=bing_list)


for key in ref_dic:
    temp_expr_matrix = []
    if len(ref_dic[key]) == 1:
        temp_expr_matrix = total_expr_df[''.join(ref_dic[key])][bing_list]
        temp_expr_matrix.name = key
        cons_expr_bing_df = cons_expr_bing_df.append(temp_expr_matrix)
    if len(ref_dic[key]) == 2:
        pre_temp_expr_matrix = total_expr_df[ref_dic[key]]
        pre_temp_expr_matrix = pre_temp_expr_matrix.loc[bing_list].values
        temp_expr_matrix = sig_corr_value_to_expr_sig(pre_temp_expr_matrix, 0.5)
        temp_expr_matrix = pd.Series(data=temp_expr_matrix, index=bing_list, name=key)
        cons_expr_bing_df = cons_expr_bing_df.append(temp_expr_matrix)
    if len(ref_dic[key]) >= 3:
        pre_temp_expr_matrix = total_expr_df[ref_dic[key]]
        pre_temp_expr_matrix = pre_temp_expr_matrix.loc[bing_list].values
        corr_matrix = matrix_to_corr_matrix(pre_temp_expr_matrix)
        sig_corr, total_corr = corr_matrix_to_sig_corr_value_and_total_corr(corr_matrix)
        temp_expr_matrix = sig_corr_value_to_expr_sig(pre_temp_expr_matrix, sig_corr)
        temp_expr_matrix = pd.Series(data=temp_expr_matrix, index=bing_list, name=key)
        cons_expr_bing_df = cons_expr_bing_df.append(temp_expr_matrix)
        
    
cons_expr_bing_df = np.round(cons_expr_bing_df, 4)

In [153]:
with gzip.open("/Users/leobrueggeman/GitHub/LINCS/consensus_expression_dataframes/cons_expr_bing_df.csv.gz", "w") as writefile:
    cons_expr_bing_df.to_csv(writefile, sep='\t')


In [247]:
cons_expr_bing_df.head()

,218075_at,218434_s_at,202852_s_at,205434_s_at,201511_at,201000_at,222064_s_at,202169_s_at,202170_s_at,210852_s_at,...,218932_at,208174_x_at,213876_x_at,204812_at,218349_s_at,204026_s_at,200808_s_at,215706_x_at,212601_at,212893_at
DB04179,-0.6729,0.1673,-0.0428,-0.5211,-0.0485,0.0550,-0.9970,-0.2953,0.0410,-0.0363,...,-0.1789,-0.5664,-0.5878,-1.3321,-0.1542,-0.0013,-0.2876,-0.1235,-0.4920,0.2758
DB01418,-0.0579,0.0570,0.2912,0.0037,0.6346,-0.2382,0.6801,0.5046,0.4870,-0.9712,...,0.1394,0.2520,0.0775,0.1062,0.4158,-0.1471,0.3563,0.2627,-0.0024,-0.1642
DB00776,0.7383,0.0341,0.0787,0.0861,0.2185,0.1826,0.0070,-0.1210,-0.1372,-0.2524,...,0.1272,0.2744,0.5430,-0.1360,0.3236,-0.2200,0.2755,-0.0945,-0.0675,-0.4155
DB01413,0.7881,0.3423,-0.4278,0.5153,-0.8084,0.4110,-0.3480,-1.0037,-0.9328,0.8399,...,-1.0454,-0.4700,-0.4232,-0.7341,0.1595,-1.8343,-0.2415,-0.2171,0.1416,-0.0659
DB01412,-0.4254,0.0274,-0.6021,-0.6224,-0.0122,-0.2788,0.0999,0.3285,0.1104,-0.3477,...,0.2984,0.2955,0.1000,0.1665,-0.4802,0.3517,-1.0559,-0.9619,-0.5808,0.1293


###Create dataframe (cons_expr_lm_df) containing consensus expression signatures of drugbank drugs by lm lists

In [210]:
# create consensus dataframe which we will fill iteratively
cons_expr_lm_df = pd.DataFrame(np.nan, index=[], columns=lm_list)

#
for key in ref_dic:
    temp_expr_matrix = []
    if len(ref_dic[key]) == 1:
        temp_expr_matrix = total_expr_df[''.join(ref_dic[key])][lm_list]
        temp_expr_matrix.name = key
        cons_expr_lm_df = cons_expr_lm_df.append(temp_expr_matrix)
        continue
    if len(ref_dic[key]) == 2:
        pre_temp_expr_matrix = total_expr_df[ref_dic[key]]
        pre_temp_expr_matrix = pre_temp_expr_matrix.loc[lm_list].values
        temp_expr_matrix = sig_corr_value_to_expr_sig(pre_temp_expr_matrix, 0.5)
        temp_expr_matrix = pd.Series(data=temp_expr_matrix, index=lm_list, name=key)
        cons_expr_lm_df = cons_expr_lm_df.append(temp_expr_matrix)
        continue
    if len(ref_dic[key]) >= 3:
        pre_temp_expr_matrix = total_expr_df[ref_dic[key]]
        pre_temp_expr_matrix = pre_temp_expr_matrix.loc[lm_list].values
        corr_matrix = matrix_to_corr_matrix(pre_temp_expr_matrix)
        sig_corr, total_corr = corr_matrix_to_sig_corr_value_and_total_corr(corr_matrix)
        temp_expr_matrix = sig_corr_value_to_expr_sig(pre_temp_expr_matrix, sig_corr)
        temp_expr_matrix = pd.Series(data=temp_expr_matrix, index=lm_list, name=key)
        cons_expr_lm_df = cons_expr_lm_df.append(temp_expr_matrix)
        continue
    
cons_expr_lm_df = np.round(cons_expr_lm_df, 4)

In [155]:
with gzip.open("/Users/leobrueggeman/GitHub/LINCS/consensus_expression_dataframes/cons_expr_lm_df.csv.gz", "w") as writefile:
    cons_expr_lm_df.to_csv(writefile, sep='\t')

In [9]:
with gzip.open("/Users/leobrueggeman/GitHub/LINCS/consensus_expression_dataframes/cons_expr_lm_df.csv.gz", "r") as readfile:
    cons_expr_lm_df = pd.DataFrame.from_csv(readfile, sep='\t')


In [10]:
cons_expr_lm_df = cons_expr_lm_df.dropna(axis=1)
cons_expr_lm_df.head()

,201000_at,209460_at,203192_at,209380_s_at,200045_at,202394_s_at,218581_at,221552_at,202123_s_at,214274_s_at,...,204937_s_at,203521_s_at,218149_s_at,212557_at,219711_at,219968_at,213196_at,218068_s_at,220661_s_at,204812_at
DB04179,0.0550,0.7874,-0.4600,-0.2522,0.3085,-0.6107,0.2423,0.3343,0.3879,-0.2264,...,-0.4299,0.0378,-1.0863,-0.7892,-1.9207,0.1297,0.3557,-0.6947,-0.3870,-1.3321
DB01418,-0.2382,-0.3098,0.2728,-0.3429,-0.3365,0.5822,0.0655,0.2286,-0.1847,0.5311,...,-1.0982,0.0400,-0.6528,-0.2476,0.3304,0.7781,-0.4427,0.5748,0.6738,0.1062
DB00776,0.2441,-0.7449,0.2914,0.5253,-0.1920,0.4492,0.7178,-0.4543,0.7310,0.1430,...,-0.0503,-0.1120,-0.8314,0.8025,-0.1992,-0.1296,0.5755,0.4058,0.3903,0.0549
DB01413,0.4110,0.7219,-1.2358,0.1712,-1.1015,-0.8303,-0.8624,4.3238,0.7965,0.5888,...,-1.5468,0.2400,1.1257,-1.3701,0.0893,0.2287,-0.2062,1.5254,0.4592,-0.7341
DB01412,-0.2788,0.4649,-0.2783,0.1116,-0.8762,-0.1364,-0.1298,0.5076,2.3611,0.9550,...,-0.8626,0.0825,1.0514,-0.2672,-0.3822,0.4752,-0.2989,-0.5640,0.1332,0.1665


### Create indication_df, which contains indications for drugbank drugs

In [11]:
# Load in indication to drugbank drug mapping
path = "https://github.com/dhimmel/indications/raw/36f0c5f143618abbbf8c9eb94b7318cb5936fd61/data/indications.tsv"
indication_df = pd.read_table(path)
indication_df = indication_df[indication_df.confidence == "high"]
indication_df.head()

,doid_code,drugbank_id,doid_name,drugbank_name,n_hc_resources,n_lc_resources,confidence
0,DOID:0050425,DB00190,restless legs syndrome,Carbidopa,1,0,high
1,DOID:0050425,DB00193,restless legs syndrome,Tramadol,1,0,high
2,DOID:0050425,DB00230,restless legs syndrome,Pregabalin,1,0,high
3,DOID:0050425,DB00268,restless legs syndrome,Ropinirole,3,0,high
4,DOID:0050425,DB00413,restless legs syndrome,Pramipexole,3,0,high


### Merge indications with landmark consensus expression signatures. Creating merger_ind_df

In [12]:
# merge indications with landmark gene expression values per drugbank drug
merger_ind_df = pd.merge(indication_df, cons_expr_lm_df, how='inner', left_on='drugbank_id', right_index=True)

In [249]:
merger_ind_df.head()

,doid_code,drugbank_id,doid_name,drugbank_name,n_hc_resources,n_lc_resources,confidence,201000_at,209460_at,203192_at,...,203521_s_at,218149_s_at,212557_at,219711_at,219968_at,213196_at,218068_s_at,220661_s_at,218916_at,204812_at
0,DOID:0050425,DB00190,restless legs syndrome,Carbidopa,1,0,high,-0.7108,0.2119,-0.5769,...,1.0220,0.9470,-0.3677,0.7610,0.9075,0.2573,-0.3911,0.6943,NaN,0.2704
116,DOID:10652,DB00190,Alzheimer's disease,Carbidopa,1,0,high,-0.7108,0.2119,-0.5769,...,1.0220,0.9470,-0.3677,0.7610,0.9075,0.2573,-0.3911,0.6943,NaN,0.2704
647,DOID:14330,DB00190,Parkinson's disease,Carbidopa,3,0,high,-0.7108,0.2119,-0.5769,...,1.0220,0.9470,-0.3677,0.7610,0.9075,0.2573,-0.3911,0.6943,NaN,0.2704
1,DOID:0050425,DB00193,restless legs syndrome,Tramadol,1,0,high,-0.6444,0.0050,1.1547,...,1.0468,0.7231,-0.2811,0.9112,0.7666,0.1982,-0.2825,0.3979,NaN,0.0211
3,DOID:0050425,DB00268,restless legs syndrome,Ropinirole,3,0,high,-0.2478,-0.0964,-0.0868,...,0.6499,-0.0834,0.5807,1.1023,-0.0519,-0.0845,-0.0905,0.7712,NaN,-0.0581


In [193]:
disease_list = set(merger_ind_df.doid_name)
for element in disease_list:
    temp_drug_list = indication_df[indication_df.doid_name == element]
    temp_drug_list = list(temp_drug_list.drugbank_id)
    temp_expr = cons_expr_lm_df[temp_drug_list]
    temp_expr = temp_expr.values
    temp_corr_matrix = matrix_to_corr_matrix(temp_expr)

    corr_len = len(corr_matrix)
    for i in range(corr_len):
        for j in range(i+1, corr_len):
            final_merged_pert_corr_list.append(['Merged',len(keep_pert_list),corr_matrix[i][j]])
    
    
    
    
    temp_df = temp_df[is_lm.pr_id]
    if len(temp_df) < 3:
        continue
    print temp_df.values
    break

[[ 0.4824  0.2545  0.6026 ...,  0.4908     nan -1.5938]
 [-2.2911  0.8529  3.1776 ...,  0.4486     nan -1.1807]
 [-2.2911  0.8529  3.1776 ...,  0.4486     nan -1.1807]
 ..., 
 [-0.3021  0.3126 -0.1893 ...,  0.1026     nan -0.346 ]
 [-0.0914 -0.4276 -0.1338 ..., -0.2063     nan  0.5104]
 [-0.1664  0.3525  0.9045 ...,  0.6702     nan -0.356 ]]


In [1]:
indication_df.head()

NameError: name 'indication_df' is not defined